# 前言

In [2]:
import torch
import numpy as np
from torch.utils.data import Dataset

**collate_fn：**即用于`collate`的`function`，用于整理数据的函数。

说到整理数据，你当然也要会用`Dataset`，因为这个你定义好后，才会产生数据嘛，产生了数据我们才能整理数据嘛，而整理数据我们使`collate_fn`。

# dataset
我们必须先看看`Dataset`如何使用

In [3]:
class Mydataset(Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):  # 必须重写
        return len(self.data)
    def __getitem__(self, item):  # 必须重写
        return self.data[item]

In [4]:
# 生成随机训练数据
a = np.random.rand(4,3)
a

array([[0.17155927, 0.1907814 , 0.98645084],
       [0.88504581, 0.96879911, 0.65166154],
       [0.60206068, 0.59390369, 0.42812672],
       [0.50019488, 0.11335474, 0.38076747]])

In [5]:
# 制作dataset
dataset = Mydataset(a)

# 调用了你上面定义的def __len__()那个函数
len(dataset)    

4

In [6]:
# 调用了你上面定义的def __getitem__()那个函数，传入的idx=0,也就是取第0个数据
dataset[0]

array([0.17155927, 0.1907814 , 0.98645084])

# 自己的理解
# dataloader之collate_fn

In [7]:
from PIL import Image
import torch
import numpy as np
from torch.utils.data import Dataset

In [8]:
class MyDataSet(Dataset):
    """自定义数据集"""

    def __init__(self, images_path):
        self.images_path = images_path

    def __len__(self):
        return len(self.images_path)

    def __getitem__(self, item):
        print("self.images_path[item] = {}".format(self.images_path[item]))
        return self.images_path[item]

    @staticmethod
    def collate_fn(batch):
        # batch 就是继承__getitem__的return
        print('batch 就是继承__getitem__的return \nbatch = {}'.format(batch))
        '''
        collate_fn的用处:
        1、自定义数据堆叠过程
        2、自定义batch数据的输出形式
        3、输入输出分别域getitem函数和loader调用时对应
        '''
        real_batch_array = np.array(batch)
        real_batch_torch = torch.from_numpy(real_batch_array)
        return real_batch_array, real_batch_torch

In [10]:
a = np.random.rand(4,3)
print("a = {}".format(a))
dataset = MyDataSet(a)
print("\nlen(dataset) = {}".format(len(dataset)))

a = [[0.99354751 0.65895154 0.67163517]
 [0.18843244 0.9580079  0.34417537]
 [0.0530706  0.15131367 0.57154965]
 [0.87065658 0.21876849 0.20610153]]

len(dataset) = 4


<font color=red>**1、不使用collate_fn自定义**</font>

In [13]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=2)

In [31]:
images = iter(dataloader)
print("\n输出__getitem__ 的return值: \n{}".format(next(images)))

self.images_path[item] = [0.99354751 0.65895154 0.67163517]
self.images_path[item] = [0.18843244 0.9580079  0.34417537]
batch 就是继承__getitem__的return 
batch = [array([0.99354751, 0.65895154, 0.67163517]), array([0.18843244, 0.9580079 , 0.34417537])]

输出__getitem__ 的return值: 
(array([[0.99354751, 0.65895154, 0.67163517],
       [0.18843244, 0.9580079 , 0.34417537]]), tensor([[0.9935, 0.6590, 0.6716],
        [0.1884, 0.9580, 0.3442]], dtype=torch.float64))


<font color=red>**2、使用collate_fn自定义**</font>

<table><tr><td bgcolor=yellow>collate_fn(batch)中的batch就是__getitem__ 的return值。整体运行流程如下：</td></tr></table>

                                                                1、先经过__init__对参数进行实例化。

                                                                2、再经过__len__返回数据集长度（数量）

                                                                3、在经过__getitem__，返回第一个batch_size的数据。

                                                                4、最后再进入collate_fn函数，collate_fn函数输入的data是__getitem__中return值。再执行__getitem__中的操作。

**collate_fn的用处:**

**1、**自定义数据堆叠过程；

**2、**自定义batch数据的输出形式；

**3、**输入输出分别域getitem函数和loader调用时对应；

In [27]:
class MyDataSet(Dataset):
    """自定义数据集"""

    def __init__(self, images_path):
        self.images_path = images_path

    def __len__(self):
        return len(self.images_path)

    def __getitem__(self, item):
        print("self.images_path[item] = {}".format(self.images_path[item]))
        return self.images_path[item]

    @staticmethod
    def collate_fn(batch):
        # batch 就是继承__getitem__的return
        print('batch 就是继承__getitem__的return \nbatch = {}'.format(batch))
        '''
        collate_fn的用处:
        1、自定义数据堆叠过程
        2、自定义batch数据的输出形式
        3、输入输出分别域getitem函数和loader调用时对应
        '''
        real_batch_array = np.array(batch)
        real_batch_torch = torch.from_numpy(real_batch_array)
        return real_batch_array, real_batch_torch

In [28]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=2, collate_fn=dataset.collate_fn)

In [30]:
images = iter(dataloader)
print("\n输出__getitem__ 的return值: \n{}".format(next(images)))

self.images_path[item] = [0.99354751 0.65895154 0.67163517]
self.images_path[item] = [0.18843244 0.9580079  0.34417537]
batch 就是继承__getitem__的return 
batch = [array([0.99354751, 0.65895154, 0.67163517]), array([0.18843244, 0.9580079 , 0.34417537])]

输出__getitem__ 的return值: 
(array([[0.99354751, 0.65895154, 0.67163517],
       [0.18843244, 0.9580079 , 0.34417537]]), tensor([[0.9935, 0.6590, 0.6716],
        [0.1884, 0.9580, 0.3442]], dtype=torch.float64))


In [22]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size = 2)

tensor([[0.6462, 0.7480, 0.0054],
        [0.7489, 0.1858, 0.4452]], dtype=torch.float64)
tensor([[0.5116, 0.3912, 0.6276],
        [0.3613, 0.4486, 0.2753]], dtype=torch.float64)


In [27]:
batch = [dataset[0],dataset[1]]
batch

[array([0.64617126, 0.74804519, 0.00543383]),
 array([0.74889795, 0.18576151, 0.44524304])]

<font color=red>`batch_size=2`即一个`batch`里面会有`2`个数据。我们以第`1`个`batch`为例，`torch.utils.data.DataLoader`会根据`dataset`取出前`2`个数据，然后弄成一个列表，如下：</font>

In [23]:
# 查看 dataloader 内部默认的collate_fn【方法一】
for step, data in enumerate(dataloader):
    images = data
    print(images)

tensor([[0.6462, 0.7480, 0.0054],
        [0.7489, 0.1858, 0.4452]], dtype=torch.float64)
tensor([[0.5116, 0.3912, 0.6276],
        [0.3613, 0.4486, 0.2753]], dtype=torch.float64)


In [25]:
# # 查看 dataloader 内部默认的collate_fn【方法二】
images = iter(dataloader)
next(images)

tensor([[0.6462, 0.7480, 0.0054],
        [0.7489, 0.1858, 0.4452]], dtype=torch.float64)

In [26]:
next(images)

tensor([[0.5116, 0.3912, 0.6276],
        [0.3613, 0.4486, 0.2753]], dtype=torch.float64)

<font color=red>然后将上面这个batch作为参数交给`collate_fn`这个函数进行进一步整理数据，然后得到`real_batch`，作为返回值。如果你不指定这个函数是什么，那么会调用`pytorch`内部的`collate_fn`。</font>

也就是说，我们如果自己要指定这个函数，`collate_fn`应该定义成下面这个样子。

In [ ]:
def my_collate(batch):#batch上面说过，是dataloader传进来的。
    #你自己定义怎么整理数据。下面会说。
    real_batch=***
    return real_batch

<font color=red>batch:</font>

[array([0.56998216, 0.72663738, 0.3706266 ]),
array([0.3403586 , 0.13931333, 0.71030221])]

<font color=red>real_batch:</font>

tensor([[0.5700, 0.7266, 0.3706],
[0.3404, 0.1393, 0.7103]], dtype=torch.float64)


**将batch变成上述real_batch很容易呀，就是把一个列表，变成了矩阵，我们也会！我们下面就来自己写一个collate_fn实现这个功能。**

In [28]:
def my_collate(batch):
    real_batch=np.array(batch)
    real_batch=torch.from_numpy(real_batch)
    return real_batch

In [29]:
dataloader2 = torch.utils.data.DataLoader(dataset, batch_size=2,collate_fn=my_collate)
# 查看 dataloader 内部默认的collate_fn【方法一】
for step, data in enumerate(dataloader2):
    images = data
    print(images)

tensor([[0.6462, 0.7480, 0.0054],
        [0.7489, 0.1858, 0.4452]], dtype=torch.float64)
tensor([[0.5116, 0.3912, 0.6276],
        [0.3613, 0.4486, 0.2753]], dtype=torch.float64)


# 应用情形
通常，我们并不需要使用这个函数，因为pytorch内部有一个默认的。但是，如果你的数据不规整，使用默认的会报错。例如下面的数据。

假设我们还是4个输入，但是维度不固定的。之前我们是每一个数据的维度都为3。

In [30]:
a=[[1,2],[3,4,5],[1],[3,4,9]]
dataset=mydataset(a,b)
dataloader=tud.DataLoader(dataset,batch_size=2)
it=iter(dataloader)
nex=next(it)
nex


NameError: name 'mydataset' is not defined

使用默认的collate_fn，直接报错，要求相同维度。

这个时候，我们可以使用自己的collate_fn，避免报错。

不过话说回来，我个人感受是：

在这里避免报错好像也没有什么用，因为大多数的神经网络都是定长输入的，而且很多的操作也要求相同维度才能相加或相乘，所以：这里不报错，后面还是报错。如果后面解决这个问题的方法是：在不足维度上进行补0操作，那么
我们为什么不在建立dataset之前先补好呢？所以，collate_fn这个东西的应用场景还是有限的。不过，明白其原理总是好事。
